In [4]:
from input_data import *

import csv
from random import random

In [13]:
def get_pin_ids():
    with open(INPUT_PINS, 'rb') as f_pins:
        reader = csv.reader(f_pins, delimiter='|')
        return [int(row[0]) for row in reader]

In [14]:
pin_ids = get_pin_ids()

print len(pin_ids), 'pin ids were loaded'

20000 pin ids were loaded


In [15]:
def get_user_ids():
    with open(INPUT_USERS, 'rb') as f_users:
        reader = csv.reader(f_users, delimiter='|')
        return [int(row[0]) for row in reader]

In [16]:
user_ids = get_user_ids()

print len(user_ids), 'user ids were loaded'

41482 user ids were loaded


In [17]:
def get_positive_repins():
    repins = []
    with open(INPUT_REPINS, 'rb') as f_repins:
        reader = csv.reader(f_repins, delimiter='|')
        for row in reader:
            repin = [int(x) for x in row]
            repins.append(repin)
        return repins

In [18]:
pos_repins = get_positive_repins()

print len(pos_repins), 'positive repins were loaded'

81447 positive repins were loaded


In [19]:
def get_all_categories():
    with open(INPUT_CATEGORIES, 'rb') as f_categories:
        reader = csv.reader(f_categories, delimiter='|')
        return [int(row[1]) for row in reader]

In [20]:
categories = get_all_categories()

print len(categories), 'categories were loaded'

32 categories were loaded


In [21]:
def generate_random_negative_repins(count):
    repins_set = set()
    for repin in pos_repins:
        repins_set.add((repin[0], repin[1]))
    
    neg_repins = []
    while len(neg_repins) < count:
        rand_user_id = user_ids[int(random() * len(user_ids))]
        rand_pin_id = pin_ids[int(random() * len(pin_ids))]
        repin = (rand_user_id, rand_pin_id)

        if repin not in repins_set:
            repin = []
            repin.append(rand_user_id)
            repin.append(rand_pin_id)
            repin.append(categories[int(random() * (len(categories) - 1))])
            repin.append(int(random() * 500))

            neg_repins.append(repin)
    return neg_repins

In [38]:
def find_near_repins(repins, repin, max_count):
    try:
        index = repins.index(repin)
        
        precision = max_count / 2;
        start_index = index - precision
        end_index = index + precision
        
        if start_index < 0:
            start_index = 0
        
        if end_index >= len(repins):
            end_index = len(repins) - 1
        
        near_repins = []
        while start_index <= end_index:
            c_repin = repins[start_index]
            
            start_index += 1
            if c_repin[0] == repin[0]:
                continue
                
            near_repins.append(c_repin)
        
        return near_repins
    except ValueError:
        return []

def generate_time_negative_repins(count):
    m_user_pins = {}
    for repin in pos_repins:
        user_id = repin[0]
        pin_id = repin[1]
        
        if user_id in m_user_pins:
            m_user_pins[user_id].add(pin_id)
        else:
            m_user_pins[user_id] = set([pin_id])
    
    neg_repins = []
    pos_repins_list = [(x[0], x[1]) for x in pos_repins]
    while len(neg_repins) < count:
        rand_user_id = user_ids[int(random() * len(user_ids))]
        
        if rand_user_id not in m_user_pins:
            continue
        
        user_pins = m_user_pins[rand_user_id]
        
        added_pins = set()
        for pin_id in user_pins:
            repin = (rand_user_id, pin_id)
            near_repins = find_near_repins(pos_repins_list, repin, 10)
            
            for repin in near_repins:
                neg_repins.append(repin)
                added_pins.add(repin[1])
        
        for pin in added_pins:
            user_pins.add(pin)
        
    return neg_repins
    
#     raise NotImplementedError()

In [34]:
def save_to_csv(data, filename):
    with open(filename, 'wb+') as f:
        writer = csv.writer(f, delimiter='|')
        for row in data:
            writer.writerow(row)

In [39]:
count = len(pos_repins)
neg_repins = generate_time_negative_repins(count)

print len(neg_repins), 'negative repins were generated'

81452 negative repins were generated


In [31]:
negative_repins_file = INPUT_NEGATIVE_REPINS
save_repins_to_csv(neg_repins, negative_repins_file)

OK
